In [1]:
"""
poetry run mlflow ui \
--backend-store-uri $MLFLOW_TRACKING_URI \
--default-artifact-root $MLFLOW_ARTIFACT_URI



poetry run mlflow ui \
--backend-store-uri $MLFLOW_TRACKING_URI \
--default-artifact-root $MLFLOW_ARTIFACT_URI \
--port 8080


Option,Rôle
poetry run,Exécute la commande mlflow ui dans l'environnement virtuel géré par Poetry.
mlflow ui,Commande pour démarrer le serveur web de l'interface utilisateur MLflow.
--backend-store-uri,"Indique l'emplacement où sont stockées les métadonnées de suivi (expérimentations, runs, métriques)."
--default-artifact-root,"Indique l'emplacement où sont stockés les fichiers d'artefacts (modèles, images, etc.)."
--port NNNN,"C'est l'option clé : elle spécifie le numéro de port (par exemple, 8080) sur lequel le serveur UI doit écouter."
"""

'\npoetry run mlflow ui --backend-store-uri $MLFLOW_TRACKING_URI --default-artifact-root $MLFLOW_ARTIFACT_URI\n\n\n\npoetry run mlflow ui --backend-store-uri $MLFLOW_TRACKING_URI --default-artifact-root $MLFLOW_ARTIFACT_URI --port 8080\n\n\nOption,Rôle\npoetry run,Exécute la commande mlflow ui dans l\'environnement virtuel géré par Poetry.\nmlflow ui,Commande pour démarrer le serveur web de l\'interface utilisateur MLflow.\n--backend-store-uri,"Indique l\'emplacement où sont stockées les métadonnées de suivi (expérimentations, runs, métriques)."\n--default-artifact-root,"Indique l\'emplacement où sont stockés les fichiers d\'artefacts (modèles, images, etc.)."\n--port NNNN,"C\'est l\'option clé : elle spécifie le numéro de port (par exemple, 8080) sur lequel le serveur UI doit écouter."\n'

In [2]:
from fertilizer_recommender.infrastructure.observability.logger import setup_logger
from loguru import logger

setup_logger(
    project_name="fertilizer_recommender",
    environment="notebook",
    level="DEBUG",
)

logger.info("Logger initialisé depuis le notebook")




2026-01-08 14:09:56 | INFO | fertilizer_recommender | notebook | 2748173838:<module>:10 | Logger initialisé depuis le notebook


In [3]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.models.baseline_logreg import BaselineLogisticRegression
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline

from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, test_df = PrepareDatasetUseCase(
    repo,
    schema,
    cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]

EXPERIMENT_NAME = "fertilizer_recommender"
RUN_NAME = generate_run_name(prefix="LOGREG")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)

def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model=BaselineLogisticRegression(
            random_state=cfg_train["project"]["seed"],
        ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "logreg_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

result = use_case.execute(
    X_df=X,
    y=y,
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print(result.fold_scores)
print(result.mean_score)

2026-01-08 14:10:01 | DEBUG | fertilizer_recommender | notebook | mlflow_setup:__init__:36 | Tracking URI : sqlite://///Users/surelmanda/Downloads/ml-projects/mlflow_central/db/mlruns.db
2026-01-08 14:10:01 | DEBUG | fertilizer_recommender | notebook | mlflow_setup:__init__:37 | Artifact URI : /Users/surelmanda/Downloads/ml-projects/mlflow_central/mlflow_artifacts


2026/01/08 14:10:01 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/08 14:10:01 INFO mlflow.store.db.utils: Updating database tables
2026/01/08 14:10:01 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/08 14:10:01 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/08 14:10:01 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/08 14:10:01 INFO alembic.runtime.migration: Will assume non-transactional DDL.


2026-01-08 14:10:01 | INFO | fertilizer_recommender | notebook | mlflow_setup:configure:53 | MLflow configuré avec succès.
2026-01-08 14:10:01 | INFO | fertilizer_recommender | notebook | mlflow_tracker:__init__:24 | MLflowExperimentTracker prêt.
2026-01-08 14:10:01 | INFO | fertilizer_recommender | notebook | experiment_tracking_service:__init__:21 | ExperimentTrackingService initialisé.
2026-01-08 14:10:01 | INFO | fertilizer_recommender | notebook | experiment_tracking_service:start_experiment:28 | Initialisation de l'expérience 'fertilizer_recommender' (run='LOGREG_2026-01-08_14h10m01s')
2026-01-08 14:10:01 | INFO | fertilizer_recommender | notebook | mlflow_tracker:setup_experiment:55 | Expérience active : fertilizer_recommender
2026-01-08 14:10:01 | INFO | fertilizer_recommender | notebook | mlflow_tracker:start_run:69 | Run démarrée : LOGREG_2026-01-08_14h10m01s
2026-01-08 14:10:01 | DEBUG | fertilizer_recommender | notebook | experiment_tracking_service:log_training_context:41 

In [4]:
logger.success(
        f"[Fold {1}] Score"
    )

2026-01-08 14:08:22 | SUCCESS | fertilizer_recommender | notebook | 646898132:<module>:1 | [Fold 1] Score


In [9]:
use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)


TypeError: TrainWithCVUseCase.__init__() got an unexpected keyword argument 'experiment_service'

In [ ]:
from fertilizer_recommender.infrastructure.tracking.run_name_generator import generate_run_name
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import ExperimentTrackingService

EXPERIMENT_NAME = "fertilizer_recommender"
RUN_NAME = generate_run_name(prefix="CATB")
MODEL_NAME = "CatBoost"
print(f"Starting experiment '{EXPERIMENT_NAME}' with run name '{RUN_NAME}'")

print("Expérience et run démarrées")
tracking_service = ExperimentTrackingService(tracker=MLflowExperimentTracker())
tracking_service.start_experiment(
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME
)

logger.info("Clôture de l'expérience MLflow")
tracking_service.close()

In [5]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config

from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.models.baseline_logreg import BaselineLogisticRegression
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline

from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, test_df = PrepareDatasetUseCase(repo, schema, cfg_train["data"]["target_col"]).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]

EXPERIMENT_NAME = "fertilizer_recommender"
MODEL_NAME = "CatBoost"
RUN_NAME = generate_run_name(prefix="CATB")
print(f"Starting experiment '{EXPERIMENT_NAME}' with run name '{RUN_NAME}'")

print("Expérience et run démarrées")
tracking_service = ExperimentTrackingService(tracker=MLflowExperimentTracker())
tracking_service.start_experiment(
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME
)

def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(schema.numeric_features, schema.categorical_features),
        model=BaselineLogisticRegression(random_state=cfg_train["project"]["seed"]),
    )

use_case = TrainWithCVUseCase(
    tracker=tracking_service,
    splitter_factory=splitter_factory,
    pipeline_factory=pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "logreg_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

Starting experiment 'fertilizer_recommender' with run name 'CATB_2026-01-08_13h09m38s'
Expérience et run démarrées
2026-01-08 13:09:38 | DEBUG | fertilizer_recommender | notebook | mlflow_setup:__init__:36 | Tracking URI : sqlite://///Users/surelmanda/Downloads/ml-projects/mlflow_central/db/mlruns.db
2026-01-08 13:09:38 | DEBUG | fertilizer_recommender | notebook | mlflow_setup:__init__:37 | Artifact URI : /Users/surelmanda/Downloads/ml-projects/mlflow_central/mlflow_artifacts
2026-01-08 13:09:38 | INFO | fertilizer_recommender | notebook | mlflow_setup:configure:53 | MLflow configuré avec succès.
2026-01-08 13:09:38 | INFO | fertilizer_recommender | notebook | mlflow_tracker:__init__:24 | MLflowExperimentTracker prêt.
2026-01-08 13:09:38 | INFO | fertilizer_recommender | notebook | experiment_tracking_service:__init__:21 | ExperimentTrackingService initialisé.
2026-01-08 13:09:38 | INFO | fertilizer_recommender | notebook | experiment_tracking_service:start_experiment:28 | Initialisat

In [7]:
tags = cfg_mlflow["mlflow"].get("tags", {})

result = use_case.execute(
    X_df=X,
    y=y,
    run_name="baseline_logreg_cv",
    params=params,
    tags=tags,
)

logger.info(f"Fold scores: {result.fold_scores}")
logger.info(f"Mean MAP@{cfg_train['training']['top_k']}: {result.mean_score:.5f}")

AttributeError: 'ExperimentTrackingService' object has no attribute 'start_run'

In [ ]:
tracking_service.start_run

In [6]:
logger.info("Clôture de l'expérience MLflow")
tracking_service.close()

2026-01-08 13:10:03 | INFO | fertilizer_recommender | notebook | 817878073:<module>:1 | Clôture de l'expérience MLflow


2026-01-08 13:10:03 | INFO | fertilizer_recommender | notebook | experiment_tracking_service:close:59 | Fermeture de l'expérience.
2026-01-08 13:10:03 | INFO | fertilizer_recommender | notebook | mlflow_tracker:end_run:75 | Fermeture de la run : d2e0225868ac4608b97edd138c9643e7
